In [1]:
# Importamos librerias
import pandas as pd

# Hacemos la lectura del csv 'retail_global_dataset'
df_retail = (
    pd.read_csv('files_folder/retail_global_dataset.csv')
    .assign(fecha = lambda x: pd.to_datetime(x['fecha'])) # Convertimos a formato de tiempo real
    .sort_values('fecha')   # Ordenar las filas del DataFrame según la columna 'fecha' en orden ascendente.
    .reset_index(drop=True) # Se está restableciendo el índice del DataFrame df a un índice numérico secuencial que empieza en 0
)
print(df_retail.head())

  id_transaccion               fecha region tipo_tienda categoria_producto  \
0     TRX-000745 2023-01-01 09:00:00   EMEA  Franquicia        Electrónica   
1     TRX-000531 2023-01-01 09:00:00   EMEA  Franquicia        Electrónica   
2     TRX-001595 2023-01-01 11:00:00  LATAM      Fisica              Hogar   
3     TRX-001446 2023-01-01 13:00:00   EMEA      Online              Hogar   
4     TRX-004218 2023-01-01 16:00:00   EMEA  Franquicia              Hogar   

   unidades_vendidas  precio_unitario  descuento_aplicado      pais  \
0                  9           493.26                0.05    España   
1                  5           203.28                0.10    España   
2                 16           247.09                0.00     Chile   
3                 15           153.23                0.00    España   
4                  5           422.40                0.10  Alemania   

   ingreso_total  
0        4217.37  
1         914.76  
2        3953.44  
3        2298.45  
4        

# Solución 1: El Peso de la Categoría (`transform`)

In [2]:
# Se calcula el porcentaje de participación de cada fila en el ingreso total de su categoría de producto respecto al total de esa categoría.
df_retail['participacion_categoria'] = df_retail['ingreso_total']/df_retail.groupby('categoria_producto')['ingreso_total'].transform('sum') * 100
print(df_retail[['ingreso_total','participacion_categoria']].head())

   ingreso_total  participacion_categoria
0        4217.37                 0.083206
1         914.76                 0.018048
2        3953.44                 0.080268
3        2298.45                 0.046666
4        1900.80                 0.038592


# Solución 2: Seguimiento de Metas por País (`cumsum`)

In [3]:
df_retail['acumulado_pais'] = df_retail.groupby('pais')['ingreso_total'].cumsum()
print(df_retail[['id_transaccion','fecha','pais','acumulado_pais']].head())

  id_transaccion               fecha      pais  acumulado_pais
0     TRX-000745 2023-01-01 09:00:00    España         4217.37
1     TRX-000531 2023-01-01 09:00:00    España         5132.13
2     TRX-001595 2023-01-01 11:00:00     Chile         3953.44
3     TRX-001446 2023-01-01 13:00:00    España         7430.58
4     TRX-004218 2023-01-01 16:00:00  Alemania         1900.80


# Solución3: Filtro de Ruido en Ventas (`rolling`)

In [4]:
# Se crea la nueva columna, aplicamos la ventana en los últimos 100 registros para sacar su promedio
df_tendencia = df_retail.assign(
    tendencia_suavizada_100 = lambda x:x['ingreso_total'].rolling(window=100).mean()
)
print("--- Tendecncias----")
print(df_tendencia[['ingreso_total','tendencia_suavizada_100']].iloc[99:103]) #Esto lo usamos para ver de la fila 99 a 102

--- Tendecncias----
     ingreso_total  tendencia_suavizada_100
99         1708.50                3686.3517
100        7027.20                3714.4500
101        7465.70                3779.9594
102        7195.43                3812.3793


# Solución 4: Alerta de Precios (`shift`)

In [5]:
df_precios = (
    df_retail
    .assign(
        precio_anterior = lambda x: x.groupby('categoria_producto')['precio_unitario'].shift(1) # Se desplaza el precio unitario una fila hacia abajo para obtener el precio anterior
    )
    .assign(
        variacion_precio_cat = lambda x: x['precio_unitario'] - x['precio_anterior'] # Se calcula la variación del precio respecto al precio anterior dentro de cada categoría
    )
)
# Revisar las primeras filas
print(df_precios[['id_transaccion','categoria_producto','precio_unitario','precio_anterior','variacion_precio_cat']].head())

  id_transaccion categoria_producto  precio_unitario  precio_anterior  \
0     TRX-000745        Electrónica           493.26              NaN   
1     TRX-000531        Electrónica           203.28           493.26   
2     TRX-001595              Hogar           247.09              NaN   
3     TRX-001446              Hogar           153.23           247.09   
4     TRX-004218              Hogar           422.40           153.23   

   variacion_precio_cat  
0                   NaN  
1               -289.98  
2                   NaN  
3                -93.86  
4                269.17  


# Hacer todo en un solo bloque de código

In [6]:
"""
# Unificamos todo en df_retail usando .assign()
df_final = (
    df_retail
    .assign(
        # Misión 1: Participación por categoría
        participacion_categoria = lambda x: x['ingreso_total'] /
                                  x.groupby('categoria_producto')['ingreso_total'].transform('sum') * 100,

        # Misión 2: Acumulado por país (YTD)
        acumulado_pais = lambda x: x.groupby('pais')['ingreso_total'].cumsum(),

        # Misión 3: Tendencia suavizada (Rolling)
        tendencia_suavizada_100 = lambda x: x['ingreso_total'].rolling(window=100).mean(),

        # Misión 4: Variación de precio (Shift)
        precio_anterior = lambda x: x.groupby('categoria_producto')['precio_unitario'].shift(1),
        variacion_precio_cat = lambda x: x['precio_unitario'] - x['precio_anterior']
    )
)

# Para verificar la Misión 4 y 3 juntas:
print("---  REPORTE FINAL DE CONTROL ---")
print(df_final[['id_transaccion', 'categoria_producto', 'precio_unitario', 'precio_anterior', 'tendencia_suavizada_100']].iloc[99:105])
"""

'\n# Unificamos todo en df_retail usando .assign()\ndf_final = (\n    df_retail\n    .assign(\n        # Misión 1: Participación por categoría\n        participacion_categoria = lambda x: x[\'ingreso_total\'] /\n                                  x.groupby(\'categoria_producto\')[\'ingreso_total\'].transform(\'sum\') * 100,\n\n        # Misión 2: Acumulado por país (YTD)\n        acumulado_pais = lambda x: x.groupby(\'pais\')[\'ingreso_total\'].cumsum(),\n\n        # Misión 3: Tendencia suavizada (Rolling)\n        tendencia_suavizada_100 = lambda x: x[\'ingreso_total\'].rolling(window=100).mean(),\n\n        # Misión 4: Variación de precio (Shift)\n        precio_anterior = lambda x: x.groupby(\'categoria_producto\')[\'precio_unitario\'].shift(1),\n        variacion_precio_cat = lambda x: x[\'precio_unitario\'] - x[\'precio_anterior\']\n    )\n)\n\n# Para verificar la Misión 4 y 3 juntas:\nprint("---  REPORTE FINAL DE CONTROL ---")\nprint(df_final[[\'id_transaccion\', \'categoria_produ